<a href="https://colab.research.google.com/github/Alirs2002/fashion_GAN/blob/main/fashion_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
import tensorflow_datasets as tfds

In [12]:
data_set = tfds.load("fashion_mnist",split= "train")

In [15]:
def scale_images(data):
  img = data["image"]
  img = img/255
  return img

In [16]:
#data_set = data_set.as_numpy_iterator()

data_set= data_set.map(scale_images)
data_set = data_set.cache()
data_set = data_set.shuffle(60000)
data_set = data_set.batch(128)
data_set = data_set.prefetch(64)

In [18]:
data_set.as_numpy_iterator().next().shape

(128, 28, 28, 1)

now lets create our generator and the discriminator models

In [22]:
from tensorflow.keras import Sequential,Model

In [23]:
def create_generator():
  model = Sequential()

  model.add(tf.keras.layers.Dense(7*7*128,input_dim = 128))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Reshape((7,7,128)))

  model.add(tf.keras.layers.UpSampling2D())
  model.add(tf.keras.layers.Conv2D(128,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.UpSampling2D())
  model.add(tf.keras.layers.Conv2D(128,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2D(128,4,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2D(128,4,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2D(1,4,padding="same",activation="sigmoid"))

  return model



bellow is a function for creating discriminator

In [38]:
def create_discriminator():

  model = Sequential()

  model.add(tf.keras.layers.Input(shape=(28,28,1)))
  model.add(tf.keras.layers.Conv2D(32,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Conv2D(64,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Conv2D(128,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Conv2D(256,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dropout(0.4))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))

  return model

In [39]:
generator = create_generator()
discriminator = create_discriminator()

now lets see a summary of the two parts of out GAN networks

In [40]:
generator.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 6272)              809088    
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 6272)              0         
                                                                 
 reshape_2 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d_4 (UpSamplin  (None, 14, 14, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 14, 14, 128)       409728    
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                      

In [41]:
discriminator.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 28, 28, 32)        0         
                                                                 
 dropout_10 (Dropout)        (None, 28, 28, 32)        0         
                                                                 
 conv2d_24 (Conv2D)          (None, 28, 28, 64)        51264     
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 28, 28, 64)        0         
                                                                 
 dropout_11 (Dropout)        (None, 28, 28, 64)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 28, 28, 128)      